# 実行環境の作成

## python3.9

In [1]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user

--2022-09-15 00:49:04--  https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  73.06M  86.5MB/s    in 0.8s    

2022-09-15 00:49:05 (86.5 MB/s) - ‘mini.sh’ saved [76607678/76607678]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py39h06a4308_2
    - cffi==1.15.0=py39hd667e15_1
    - charset-normalizer=

In [2]:
!python --version

Python 3.9.12


## Google Cloud Storage

下記コードでGCPに接続

In [3]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [5]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/15 00:42:55.008447 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/15 00:42:55.021886 Opening GCS connection...
2022/09/15 00:42:55.487174 Mounting file system "statistics-hyogo"...
2022/09/15 00:42:55.487876 File system has been successfully mounted.


## Google Secret Maneger

ライブラリをインストール

In [ ]:
pip install google-cloud-secret-manager

シークレットを呼び出す関数

In [ ]:
from google.cloud import secretmanager

def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

ESTAT_APPIDを取得する

In [ ]:
PROJECT_ID = 'primal-buttress-342908'
ESTAT_APPID = access_secret(PROJECT_ID,'ESTAT_APPID' )
# print(ESTAT_APPID)

In [4]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [5]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

統計表を指定してメタ情報を取得する

[API仕様](https://www.e-stat.go.jp/api/api-info/e-stat-manual3-0#api_2_2)

'statsDataId'に統計表コードを指定

In [6]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [7]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
print(meta)

{'GET_META_INFO': {'RESULT': {'STATUS': 0, 'ERROR_MSG': '正常に終了しました。', 'DATE': '2022-09-15T09:50:30.889+09:00'}, 'PARAMETER': {'LANG': 'J', 'STATS_DATA_ID': '0000010101', 'DATA_FORMAT': 'J'}, 'METADATA_INF': {'TABLE_INF': {'@id': '0000010101', 'STAT_NAME': {'@code': '00200502', '$': '社会・人口統計体系'}, 'GOV_ORG': {'@code': '00200', '$': '総務省'}, 'STATISTICS_NAME': '都道府県データ 基礎データ', 'TITLE': {'@no': '0000010101', '$': 'Ａ\u3000人口・世帯'}, 'CYCLE': '年度次', 'SURVEY_DATE': 0, 'OPEN_DATE': '2022-03-04', 'SMALL_AREA': 0, 'COLLECT_AREA': '全国', 'MAIN_CATEGORY': {'@code': '99', '$': 'その他'}, 'SUB_CATEGORY': {'@code': '99', '$': 'その他'}, 'OVERALL_TOTAL_NUMBER': 500976, 'UPDATED_DATE': '2022-03-04', 'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '都道府県データ', 'TABULATION_SUB_CATEGORY1': '基礎データ'}, 'DESCRIPTION': {'TABULATION_CATEGORY_EXPLANATION': '社会・人口統計体系の都道府県ごとに集計したデータを提供します。'}, 'TITLE_SPEC': {'TABLE_NAME': 'Ａ\u3000人口・世帯'}}, 'CLASS_INF': {'CLASS_OBJ': [{'@id': 'tab', '@name': '観測値', 'CLASS': {'@code': '00001', 

## カテゴリコード一覧

カテゴリコード'cat01'のリストを作成する関数

In [8]:
def get_estat_catefory_codes(statsDataId):
  meta= get_estat_meta(statsDataId)
 
  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # カテゴリ一覧
  categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # カテゴリコード一覧
  catagory_codes = [d.get('@code') for d in categories]

  return catagory_codes

category_codes = get_estat_catefory_codes(statsDataId)
print(category_codes)
# print(len(category_codes))

['A1101', 'A110101', 'A110102', 'A1102', 'A110201', 'A110202', 'A1201', 'A120101', 'A120102', 'A1202', 'A120201', 'A120202', 'A1203', 'A120301', 'A120302', 'A1204', 'A120401', 'A120402', 'A1205', 'A120501', 'A120502', 'A1206', 'A120601', 'A120602', 'A1207', 'A120701', 'A120702', 'A1208', 'A120801', 'A120802', 'A1209', 'A120901', 'A120902', 'A1210', 'A121001', 'A121002', 'A1211', 'A121101', 'A121102', 'A1212', 'A121201', 'A121202', 'A1213', 'A121301', 'A121302', 'A1214', 'A121401', 'A121402', 'A1215', 'A121501', 'A121502', 'A1216', 'A121601', 'A121602', 'A1217', 'A121701', 'A121702', 'A1218', 'A121801', 'A121802', 'A1219', 'A121901', 'A121902', 'A1220', 'A122001', 'A122002', 'A1221', 'A122101', 'A122102', 'A1231', 'A1301', 'A130101', 'A130102', 'A1302', 'A130201', 'A130202', 'A1303', 'A130301', 'A130302', 'A1304', 'A130401', 'A130402', 'A1305', 'A130501', 'A130502', 'A1306', 'A130601', 'A130602', 'A1401', 'A140101', 'A140102', 'A1402', 'A140201', 'A140202', 'A1403', 'A140301', 'A140302'

## 地域コード一覧

地域コード'area'のリスト一覧を作成する関数

全国'00000'は除外している

In [9]:
def get_estat_area_codes(statsDataId):
  # estat-APIのメタ情報を取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # 地域一覧
  areas = next((d for d in CLASS_OBJ if d['@id'] == 'area'), None)['CLASS']

  # 地域コード一覧（全国'00000'は削除）
  area_codes = [d.get('@code') for d in areas]
  area_codes.remove('00000')

  return area_codes

area_codes = get_estat_area_codes(statsDataId)
print(area_codes)

['01000', '02000', '03000', '04000', '05000', '06000', '07000', '08000', '09000', '10000', '11000', '12000', '13000', '14000', '15000', '16000', '17000', '18000', '19000', '20000', '21000', '22000', '23000', '24000', '25000', '26000', '27000', '28000', '29000', '30000', '31000', '32000', '33000', '34000', '35000', '36000', '37000', '38000', '39000', '40000', '41000', '42000', '43000', '44000', '45000', '46000', '47000']


# estat-APIのデータを取得

まずは、総人口のデータを取得してみる

cat01に'A1101'を指定
areaは全都道府県を指定

リストを指定する場合はカンマ区切りの文字列に変換する

## estat-APIのデータ取得

In [10]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1231'
params['cdArea']=",".join(area_codes)

# print (params)

estat-APIのデータを取得する関数を作成

In [11]:
def get_estat_data(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

res = get_estat_data(params)
# print(res)

## pandas DataFrameに変換

estat-APIのレスポンスからVALUEを抜き出し、DataFrameに変換する

In [12]:
res = get_estat_data(params)

# VALUE
VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df_value = pd.json_normalize(VALUE)
df_value

,@tab,@cat01,@area,@time,@unit,$
0,00001,A1231,01000,2010100000,歳,47.7
1,00001,A1231,01000,2015100000,歳,49.7
2,00001,A1231,02000,2010100000,歳,49.1
3,00001,A1231,02000,2015100000,歳,51.4
4,00001,A1231,03000,2010100000,歳,49.5
...,...,...,...,...,...,...
89,00001,A1231,45000,2015100000,歳,50.2
90,00001,A1231,46000,2010100000,歳,49.0
91,00001,A1231,46000,2015100000,歳,50.6
92,00001,A1231,47000,2010100000,歳,40.0


カテゴリ情報は'CLASS_OBJ'に格納されている。

これもDataFrameに変換

In [ ]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

df_categories = pd.json_normalize(categories)
df_categories

統計表情報は'TABLE_INF'に格納されている

In [ ]:
# TABLE_INF
TABLE_INF = res['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

statDataId = TABLE_INF['@id']
statDataName = TABLE_INF['STAT_NAME']['$']

print(statDataId,statDataName)

estat-APIのレスポンスをひとつのDataFrameにまとめる関数を作成

In [13]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

df_org = conv_estat_response_to_dataframe(res)
df_org

,@tab,@cat01,@area,@time,@unit,$,@tab_name,@cat01_name,@area_name,@time_name,statsDataId,statsDataName
0,00001,A1231,01000,2010100000,歳,47.7,観測値,A1231_年齢中位数,北海道,2010年度,0000010101,社会・人口統計体系
1,00001,A1231,02000,2010100000,歳,49.1,観測値,A1231_年齢中位数,青森県,2010年度,0000010101,社会・人口統計体系
2,00001,A1231,03000,2010100000,歳,49.5,観測値,A1231_年齢中位数,岩手県,2010年度,0000010101,社会・人口統計体系
3,00001,A1231,04000,2010100000,歳,44.8,観測値,A1231_年齢中位数,宮城県,2010年度,0000010101,社会・人口統計体系
4,00001,A1231,05000,2010100000,歳,52.4,観測値,A1231_年齢中位数,秋田県,2010年度,0000010101,社会・人口統計体系
...,...,...,...,...,...,...,...,...,...,...,...,...
133,NaN,NaN,NaN,2016100000,NaN,NaN,NaN,NaN,NaN,2016年度,0000010101,社会・人口統計体系
134,NaN,NaN,NaN,2017100000,NaN,NaN,NaN,NaN,NaN,2017年度,0000010101,社会・人口統計体系
135,NaN,NaN,NaN,2018100000,NaN,NaN,NaN,NaN,NaN,2018年度,0000010101,社会・人口統計体系
136,NaN,NaN,NaN,2019100000,NaN,NaN,NaN,NaN,NaN,2019年度,0000010101,社会・人口統計体系


## DataFrameの整形

使い勝手が良くなるように、DataFrameを整形していく

必要な列だけ抽出する

In [ ]:
df_1 = df_org[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

# print(df_1)

列名を変更する

In [ ]:
columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
df_1 = df_1.rename(columns=columns)

# print(df_1)

欠損値が含まれる行を削除する

In [ ]:
df_1 = df_1.dropna()

# print(df_1)

categoryNameから不要な情報（categoryCode）を削除する

In [ ]:
df_1['categoryName'] = df_1.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

# print(df_1)

timeCodeを4桁文字列に置換する

In [ ]:
df_1['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

# print(df_1)

ここまでを関数にまとめる

In [15]:
def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
  df_res = df_res.rename(columns=columns)

  # 欠損データ削除
  df_res = df_res.dropna()

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)

  return df_res

df_fmt = format_estat_dataframe(df_org)
df_fmt

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,areaName,value,unit
0,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,01000,北海道,47.7,歳
1,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,02000,青森県,49.1,歳
2,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,03000,岩手県,49.5,歳
3,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,04000,宮城県,44.8,歳
4,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,05000,秋田県,52.4,歳
...,...,...,...,...,...,...,...,...,...,...
89,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,43000,熊本県,49.0,歳
90,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,44000,大分県,50.2,歳
91,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,45000,宮崎県,50.2,歳
92,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,46000,鹿児島県,50.6,歳


## 順位を付与する

年度ごとに順位を付与

In [16]:
def format_estat_dataframe_withrank(df_arg):

  # 年度リストを作成
  times = df_arg['timeCode'].tolist()
  times = list(set(times))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 年度毎に順位を付与
  for time in times:

    # 単年度のdataを抽出
    df_time = df_arg.copy()[df_arg['timeCode'] == time]

    # valueを数値に変換して、'rank'を付与
    df_time = df_time.astype({'value': float})
    df_time['rank'] = df_time.rank(ascending=False)['value'].astype(int).astype(str)

    # 結合
    df_res = pd.concat([df_res, df_time])
    df_res = df_res.astype({'value': str})

  return df_res

df_last = format_estat_dataframe_withrank(df_fmt)
df_last

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,areaName,value,unit,rank
47,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,01000,北海道,49.7,歳,18
48,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,02000,青森県,51.4,歳,5
49,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,03000,岩手県,51.3,歳,6
50,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,04000,宮城県,46.7,歳,37
51,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,05000,秋田県,54.8,歳,1
...,...,...,...,...,...,...,...,...,...,...,...
42,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,43000,熊本県,47.8,歳,19
43,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,44000,大分県,48.8,歳,13
44,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,45000,宮崎県,48.6,歳,15
45,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,46000,鹿児島県,49.0,歳,10


## まとめ

paramsを指定してから、順位を付与したDataFrameを取得するまでを関数にまとめる

In [17]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1101'
params['cdArea']=",".join(area_codes)

In [18]:
def get_estat_dataframe(params):
  # estat-APIのレスポンス取得
  res = get_estat_data(params)

  # レスポンスをDataFrameに変換
  df = conv_estat_response_to_dataframe(res)

  # DataFrameを整形
  df = format_estat_dataframe(df)

  # 順位を付与
  df = format_estat_dataframe_withrank(df)

  return df

df = get_estat_dataframe(params)
df

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,areaName,value,unit,rank
1504,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,01000,北海道,5579000.0,人,8
1505,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,02000,青森県,1409000.0,人,29
1506,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,03000,岩手県,1364000.0,人,32
1507,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,04000,宮城県,2354000.0,人,15
1508,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,05000,秋田県,1121000.0,人,37
...,...,...,...,...,...,...,...,...,...,...,...
700,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,43000,熊本県,1841000.0,人,22
701,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,44000,大分県,1240000.0,人,32
702,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,45000,宮崎県,1172000.0,人,36
703,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,46000,鹿児島県,1805000.0,人,23


In [19]:
pip install pandas_gbq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'primal-buttress-342908.esat_api'
table_id='test3'


df = get_estat_dataframe(params)
# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="append")

GenericGBQException: ignored

# 統計表に含まれる全てのデータをDataframeに変換する

'statsDataId'で統計表を指定して、含まれる統計データを一括でDataFrameに変換する

In [ ]:
statsDataId = '0000010101'

In [ ]:
def get_estat_dataframe_all(statsDataId):

  # 統計表のメタ情報を取得
  meta = get_estat_meta(statsDataId)

  # カテゴリ一覧
  category_codes = get_estat_catefory_codes(statsDataId)

  # 地域コード一覧
  area_codes = get_estat_area_codes(statsDataId)

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  for category_code in category_codes:
    # リクエストパラメータ生成
    params = {}
    params['statsDataId'] = statsDataId
    params['cdCat01'] = category_code
    params['cdArea']=",".join(area_codes)

    # DataFrameを取得
    print(category_code)
    df_category = get_estat_dataframe(params)

    df_res = pd.concat([df_res, df_category])

  # return df_res
  return 'test'


df = get_estat_dataframe_all(statsDataId)
# df


# 新しいセクション